This script pulls in MLS player salary info for the 2018-2019 season via https://mlsplayers.org/resources/salary-guide

In [5]:
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd

In [13]:
with open("MLS_SALARIES.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [14]:
def scrape():
    teams = {}
    players = soup.findAll("tr")
    table = pd.DataFrame(columns={'first_name', 'last_name', 'team', 'position', 'base_pay', 'total_pay'})
    player_list = []
    for player in players[1:]:
        attributes = player.findAll("td")
        player_list.append({
            'first_name': str(attributes[0])[4:-5],
            'last_name': str(attributes[1])[4:-5],
            'team': str(attributes[2])[4:-5],
            'position': str(attributes[3])[4:-5],
            'base_pay': Decimal(str(attributes[4])[5:-5].replace(',', '')),
            'total_pay': Decimal(str(attributes[5])[5:-5].replace(',', ''))
        })
    return pd.DataFrame(player_list)

In [75]:
full_players = scrape()

In [79]:
temp_teams = {}
for index, player in full_players.iterrows():
    if player['team'] in temp_teams:
        temp_teams[player['team']]['total_base'] += player['base_pay']
        temp_teams[player['team']]['total_comp'] += player['total_pay']
        temp_teams[player['team']]['player_count'] += 1
    else:
        temp_teams[player['team']] = {
            'team': player['team'],
            'total_base': player['base_pay'],
            'total_comp': player['total_pay'],
            'player_count': 1
        }

In [88]:
teams = pd.DataFrame(temp_teams).transpose()

In [101]:
teams = teams.reset_index()

In [103]:
teams.loc[teams['player_count'].idxmax()]

TypeError: reduction operation 'argmax' not allowed for this dtype

In [107]:
teams

,index,player_count,team,total_base,total_comp
0,Sporting Kansas City,29,Sporting Kansas City,11132768.52,11554237.01
1,FC Dallas,34,FC Dallas,8219481.52,9334556.52
2,LAFC,29,LAFC,11824368.73,14134135.99
3,Atlanta United,32,Atlanta United,10669120.49,11606330.49
4,Houston Dynamo,30,Houston Dynamo,5490429.93,5896563.27
5,San Jose Earthquakes,30,San Jose Earthquakes,7667242.28,8308343.41
6,New York Red Bulls,31,New York Red Bulls,7353490.28,8064992.45
7,Columbus Crew,29,Columbus Crew,7285837.47,7715954.14
8,LA Galaxy,27,LA Galaxy,14799179.79,17507008.30
9,Portland Timbers,31,Portland Timbers,10543917.96,10994792.96
